# Building the UI
In addition to the Neo4j console and the notebook earlier, we can provide develop a public facing website that we can share with the public without providing them write access to the neo4j database.

The UI provides single and two-term search where the latter uses allShortestPath to query all the shortest paths between two terms. To build the UI, we need to create a UI schema that is built using the data that was ingested in the graph db. This notebook automates the creation of this schema file.

In [7]:
import re
import json
from glob import glob
import pandas as pd

## Node Schema

```
{
    "node": "Gene",
    "example": [
        "COPB2",
        "SDF2L1"
    ],
    "display": [
        {
            "label": "Gene ID",
            "text": "${id}",
            "type": "text"
        },
        {
            "label": "label",
            "text": "${label}",
            "type": "text"
        },
        {
            "label": "uri",
            "text": "${uri}",
            "href": "${uri}",
            "type": "link"
        }
    ],
    "search": [
        "id",
        "label"
    ]
}

```
This is a node schema for the node "Gene". The example field will display as an example on the UI while the display field tells the UI what to display as a tooltip. Meanwhile, search tells us what property to perform search on

In [8]:
node_schema = []
node_pattern = "(?P<directory>.+)/(?P<label>.+)\.(?P<entity>.+)\.csv"
for filename in glob("csv/*.nodes.csv"):
    match = re.match(node_pattern, filename).groupdict()
    label = match["label"].replace("_", " ")
    df = pd.read_csv(filename)
    display = []
    for i in df.columns:
        if i == "id":
            display.append({
                'label': '%s ID'%label,
                'text': "${%s}"%i,
                'type': 'text'
            })
        else:
            val = str(df.at[0, i])
            if val.startswith("http"):
                display.append({
                    'label': i,
                    'text': "${%s}"%i,
                    'href': "${%s}"%i,
                    'type': 'link'
                })
            else:
                display.append({
                    'label': i,
                    'text': "${%s}"%i,
                    'type': 'text'
                })
    examples = list(df.head(2).label)
    node = {
        "node": label,
        "example": examples,
        "display": display,
        "search": [
            "id",
            "label"
        ]
    }
    node_schema.append(node)
        

## Edge Schema
```
{
    "match": [
        "KEGG"
    ],
    "selected": true,
    "display": [
        {
            "label": "source",
            "text": "${source}",
            "type": "text"
        },
        {
            "label": "relation",
            "text": "${relation}",
            "type": "text"
        },
        {
            "label": "target",
            "text": "${target}",
            "type": "text"
        },
        {
            "label": "source label",
            "text": "${source_label}",
            "type": "text"
        },
        {
            "label": "target label",
            "text": "${target_label}",
            "type": "text"
        },
        {
            "label": "resource",
            "text": "${resource}",
            "href": "${resource}",
            "type": "link"
        }
    ]
}
```

This is the same with node schema with a few differences, first match allows you to match multiple edge types and selected is a field that allows us to select what edges are queried by default when doing two term searches.

In [9]:
edge_schema = []
edge_pattern = "(?P<directory>.+)/(?P<source_type>.+)\.(?P<relation>.+)\.(?P<target_type>.+)\.(?P<entity>.+)\.csv"

for filename in glob("csv/*.edges.csv"):
   match = re.match(edge_pattern, filename).groupdict()
   entity = match["entity"]
   source_type = match["source_type"]
   relation = match["relation"].replace("_", " ")
   edge = {
      "match": [relation],
      "selected": True,
   }
   df = pd.read_csv(filename)
   display = []
   for i in df.columns:
      if i == "id":
         display.append({
               'label': '%s ID'%label,
               'text': "${%s}"%i,
               'type': 'text'
         })
      else:
         val = str(df.at[0, i])
         if val.startswith("http"):
               display.append({
                  'label': i.replace("_", " "),
                  'text': "${%s}"%i,
                  'href': "${%s}"%i,
                  'type': 'link'
               })
         else:
               display.append({
                  'label': i.replace("_", " "),
                  'text': "${%s}"%i,
                  'type': 'text'
               })
         edge["display"] = display
   edge_schema.append(edge)
         

## Additional schema

The next part are miscellaneous schemas that you can use to modify our UI

In [10]:
rest = {
    "header": {
		"title": "Enrichr Knowledge Graph",
		"icon": {
			"src": "https://minio.dev.maayanlab.cloud/enrichr-kg/enrichr-kg.png",
			"favicon": "https://minio.dev.maayanlab.cloud/enrichr-kg/enrichr-kg.png",
			"alt": "Enrichr logo",
			"key": "enrichr_logo",
			"width": 100,
			"height": 60
		},
		"tabs": [
			{
				"endpoint": "/",
				"label": "Term & Gene Search",
				"type": "page",
				"component": "KnowledgeGraph",
				"props": {
					"init_function": "initialize_kg"
				}
			}
		]
	},
	"footer": [
		{
			"type": "github",
			"code": "https://github.com/MaayanLab/enrichr-kg",
			"issues": "https://github.com/MaayanLab/enrichr-kg/issues"
		},
		{
			"type": "icon",
			"key": "lab_icon",
			"src": "https://minio.dev.maayanlab.cloud/enrichr-kg/maayan-lab.jpg",
			"alt": "MaayanLab",
			"href": "https://labs.icahn.mssm.edu/maayanlab/",
			"height": 125,
			"width": 150
		},
		{
			"type": "icon",
			"key": "center_icon",
			"src": "https://minio.dev.maayanlab.cloud/enrichr-kg/ISMMS_Center_Bioinformatics_RGB_Vrtl.jpg",
			"alt": "COB",
			"href": "https://icahn.mssm.edu/research/bioinformatics",
			"height": 125,
			"width": 200
		}

	]
}

## Putting it all together

In [11]:
schema = {
    "nodes": node_schema,
    "edges": edge_schema,
    **rest
}

with open("../schema.json", 'w') as o:
    o.write(json.dumps(schema, indent=4))